<a href="https://colab.research.google.com/github/YuehMintTai/GAETest20210722/blob/main/%E4%B8%89%E7%B8%BD%E5%8C%97%E6%8A%95%E9%96%80%E8%A8%BA%E5%8A%A9%E6%89%8B6_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask_ngrok
!pip install line-bot-sdk
!pip install bs4
!pip install requests

In [13]:
from flask_ngrok import (run_with_ngrok)
from flask import (Flask, request, abort)
from linebot import (LineBotApi, WebhookHandler)
from linebot.exceptions import (InvalidSignatureError)
from linebot.models import *
from bs4 import BeautifulSoup
import requests
import OPDAid6_SN
import re
import myBubbles
import json

In [26]:
from flask_ngrok import (run_with_ngrok)
from flask import (Flask, request, abort)
from linebot import (LineBotApi, WebhookHandler)
from linebot.exceptions import (InvalidSignatureError)
from linebot.models import *
from bs4 import BeautifulSoup
import requests
import OPDAid6_SN
import re
import myBubbles
##myBubble_OPDs出問題,要修正它...
app=Flask(__name__)
run_with_ngrok(app)

line_bot_api=LineBotApi(OPDAid6_SN.myLineBotApi)
handler=WebhookHandler(OPDAid6_SN.myWebhookHandler)
myID=OPDAid6_SN.myUserID
line_bot_api.push_message(myID,FlexSendMessage(
alt_text='三總北投門診助手6.0',
contents=myBubbles.myBubble_greeting()))
#######OPDLists############
OPDLists=[]
OPDLinks=[]
target_url='https://wwwv.tsgh.ndmctsgh.edu.tw/unit/30000/28387'
responses1=requests.get(target_url)
soup=BeautifulSoup(responses1.text,'html.parser')
myDiv=soup.find(id='html_text')
myList=myDiv.find_all('a')
for tag in myList:
  try:
    OPDLists.append(tag.find('u').contents[0])
    OPDLinks.append(tag.get('href'))
  except:
    OPDLists.append(tag.contents[0])
    OPDLinks.append('https://wwwv.tsgh.ndmctsgh.edu.tw{}'.format(tag.get('href')))

def myBubble_OPD1():
  myContents_opd=[]
  i=0
  for i in range(len(OPDLists)):
    myContent=ButtonComponent(style='link',height='sm',action=URIAction(label=OPDLists[i],uri=OPDLinks[i]))
    myContents_opd.append(myContent)
  myBubble_opd=BubbleContainer(
      body=BoxComponent(layout='vertical',spacing='sm',contents=myContents_opd))
  return myBubble_opd

def myBubble_Director():
  myContent_dir=ButtonComponent(style='link',height='sm',action=URIAction(label='楊斯年院長',uri='https://wwwv.tsgh.ndmctsgh.edu.tw/DocDet/194/30000/25075/1766'))
  myBubble_dir=FlexSendMessage(
      alt_text='楊斯年院長',
      contents=FlexSendMessage.BubbleContainer(
          direction='ltr'
          hero=FlexSendMessage.ImageComponent(
              url='https://wwwv.tsgh.ndmctsgh.edu.tw/files/web/194/doctor/30000/25075/%E7%B2%BE%E7%A5%9E%E7%A7%91-%E6%A5%8A%E6%96%AF%E5%B9%B4.jpg',
              size='full',
              aspect_ratio='2;1',
              aspect_mode='cover',
              action=actions.URIAction(uri='https://wwwv.tsgh.ndmctsgh.edu.tw/DocDet/194/30000/25075/1766',label='楊斯年院長')
          )
      )
  )
  
  return myBubble_dir

@app.route('/')
def index():
  return '歡迎使用三總北投門診助手6.0'

@app.route('/callback',methods=['POST'])
def callback():
  signature=request.headers['X-Line-Signature']
  body=request.get_data(as_text=True)
  try:
    handler.handle(body,signature)
  except InvalidSignatureError:
    abort(400)
  return 'OK'

@handler.add(MessageEvent,message=TextMessage)
def handle_message(event):
  message=text=event.message.text
  if re.match('門診時間',message):
    line_bot_api.reply_message(event.reply_token, FlexSendMessage(alt_text=message,contents=myBubble_OPD1()))
  elif re.match('費用查詢',message):
    line_bot_api.reply_message(event.reply_token, FlexSendMessage(alt_text=message,contents=myBubbles.myBubble_Fee()))
  elif re.match('重大公告',message):
    line_bot_api.reply_message(event.reply_token, FlexSendMessage(alt_text=message,contents=myBubbles.myBubble_News()))
  else:
    #line_bot_api.push_message(myID,FlexSendMessage(alt_text='你好',contents=myBubbles.myBubble_greeting()))
    line_bot_api.push_message(myID,FlexSendMessage(alt_text='你好',contents=myBubble_Director()))
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://3a37-34-122-70-23.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [12/Sep/2021 09:08:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Sep/2021 09:08:40] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [12/Sep/2021 09:09:09] "POST /callback HTTP/1.1" 200 -
[2021-09-12 09:09:15,380] ERROR in app: Exception on /callback [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.7/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py",

In [ ]:
##Run with flask_ngrok import myBubbles
app=Flask(__name__)
run_with_ngrok(app)

line_bot_api=LineBotApi(OPDAid6_SN.myLineBotApi)
handler=WebhookHandler(OPDAid6_SN.myWebhookHandler)
myID=OPDAid6_SN.myUserID
line_bot_api.push_message(myID,FlexSendMessage(
alt_text='三總北投門診助手6.0',
contents=myBubbles.myBubble_greeting()))

@app.route('/')
def index():
    return '歡迎使用三總北投門診助手6.0'

@app.route('/callback',methods=['POST'])
def callback():
    signature=request.headers['X-Line-Signature']
    body=request.get_data(as_text=True)
    try:
        handler.handle(body,signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'


@handler.add(MessageEvent,message=TextMessage)
def handle_message(event):
    message=text=event.message.text
    if re.match('門診時間',message):
      line_bot_api.reply_message(event.reply_token, FlexSendMessage(alt_text=message,contents=myBubbles.myBubble_OPDs(message)))
    elif re.match('醫師查詢',message):
      line_bot_api.reply_message(event.reply_token, FlexSendMessage(alt_text=message,contents=myBubbles.myBubble_doctors(message)))
    elif re.match('重大公告',message):
      line_bot_api.reply_message(event.reply_token, FlexSendMessage(alt_text=message,contents=myBubbles.myBubble_News(message)))
    elif re.match('費用查詢',message):
      line_bot_api.reply_message(event.reply_token, FlexSendMessage(alt_text=message,contents=myBubbles.myBubble_Fee('123')))
    else:
      line_bot_api.push_message(myID,FlexSendMessage(alt_text='你好',contents=myBubbles.myBubble_greeting()))
      ##line_bot_api.reply_message(event.reply_token,FlexSendMessage(alt_text=message,contents=myBubbles.myBubble_greeting()))

##if __name__=='__main__':
app.run()